# Modelos de Recomendação Baseados em Similaridade de Conteúdo

In [1]:
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy.spatial.distance import cdist
from IPython.display import display
from sklearn.metrics import davies_bouldin_score, silhouette_score
import sys
sys.path.append('../auxiliarScripts')
from dataset_reader import SpotifyPlaylistDataset
from yellowbrick.cluster import KElbowVisualizer

In [2]:
# 1001 -> ambiente

### Carrega e Normaliza os Dados

In [3]:
tracks = pd.read_csv('../tracks/tracks.csv', index_col='id')
tracks

,name,duration_ms,artists,explicit,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature
id,,,,,,,,,,,,,,,,
spotify:track:0hJkacKTAr8GZeKwcPuUk2,Yesterday's Trash,52213,"{""(24OGO7fr96L0azkG3pZZa2,\""Modern Life Is War...",t,0,0.004530,0.324,0.971,0.012800,0,0.3340,-3.168,1,0.0548,95.721,4
spotify:track:1xjFMVi8pn5VIWiqrdxAMc,promise [interlude],88888,"{""(5b0XQwyoJBFd3MwL2YxEPO,Nohidea)""}",f,0,0.833000,0.545,0.495,0.629000,11,0.3860,-13.538,1,0.1020,62.631,4
spotify:track:2ahiPTFTfIPs7UKqjKhJkc,Requiem - Alex Vidal Remix,564720,"{""(6WrPC9g41hy7LUnpxdy6u7,\""Alex Vidal\"")"",""(2...",f,0,0.033200,0.695,0.592,0.892000,0,0.5790,-9.588,1,0.0345,125.008,4
spotify:track:2z5xCMFxAo7dn6zsu6UwL9,Kaké,245356,"{""(44DhAhLWsSFvSqVXDUruVE,\""Young Paris\"")""}",t,0,0.000440,0.450,0.629,0.895000,7,0.1080,-7.059,1,0.1460,193.228,3
spotify:track:16axQgz02GFMiU2xKMT4qJ,F the Bull$h1t - Original Mix,315773,"{""(2ohlvFf9PBsDELdRstPtlP,\""Ferry Corsten\"")""}",f,0,0.000927,0.846,0.707,0.903000,6,0.0666,-5.949,0,0.1270,127.970,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,Waking Up The Dawn,294333,"{""(7beSXA7nBwZ5xJ8jUa77ZL,\""Michael English\"")""}",f,2,0.109000,0.538,0.609,0.000006,5,0.0948,-7.392,1,0.0275,87.960,4
spotify:track:1nvTse7nkCr8tO8LOAell4,Here Comes the Star,202733,"{""(48YxSlb23RAaCd4RyHcV9V,\""Herman's Hermits\"")""}",f,0,0.698000,0.339,0.325,0.000005,2,0.5550,-11.445,1,0.0252,91.286,4
spotify:track:5vKVjyFJNtfdHXzzLaWbpz,You're With Me,293546,"{""(0WTXG5dSB0wpykAqPpEmFl,UnchainD)""}",f,0,0.018600,0.482,0.443,0.000345,4,0.1100,-8.622,1,0.0276,144.962,4


#### Para pegar uma música: tracks.loc[id]
#### Para pesquisar o nome de uma música e pegar seu id: tracks.index[tracks['name'] == "nome música"]

#### Deixa apenas as features que serão utilizadas e separa um dataframe com nome e artistas para poderem ser retornados no final

In [4]:
tracks[tracks['name'] == 'Fantasia in D Minor, K.397']

,name,duration_ms,artists,explicit,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature
id,,,,,,,,,,,,,,,,
spotify:track:3ekVOvu8iE33gEGLDq9lxJ,"Fantasia in D Minor, K.397",436560,"{""(4NJhFmfw43RLBLjQvxDuRS,\""Wolfgang Amadeus M...",f,15,0.993,0.392,0.0249,0.946,2,0.114,-28.685,1,0.0632,134.872,4
spotify:track:2H80pfVM3uQQCdb86hmhxf,"Fantasia in D Minor, K.397",324186,"{""(4NJhFmfw43RLBLjQvxDuRS,\""Wolfgang Amadeus M...",f,31,0.973,0.393,0.1460,0.786,7,0.114,-25.025,1,0.0438,128.803,4
spotify:track:5NEQySJDwnf3O7bISjnQ6q,"Fantasia in D Minor, K.397",316653,"{""(4NJhFmfw43RLBLjQvxDuRS,\""Wolfgang Amadeus M...",f,0,0.994,0.385,0.0499,0.906,2,0.107,-25.262,1,0.0543,125.897,4


In [5]:
# Set de músicas de teste
classical_music = 'spotify:track:4Znak3UnsXjRYQDSK0oP6V' # (clássica) Symphony No.5 in C minor, Op.67 | Beethoven

rap_music = 'spotify:track:5Tou8lYopj9qCIFBFBomZ5' # (rap) Yoshi City | Yung Lean

rap_music2 = 'spotify:track:15TJBhID2NsMMp2etO0B03' # (rap) Wrist Cry | Bladee

anime_music = 'spotify:track:0zFRHGYU1zP6q45qbils3z' # (anime) Go!!! | Flow

rock_music = 'spotify:track:2hmibAtdObO8F4tnhLENuQ' # (rock) Reptilia | The Strokes

eletronic_music = 'spotify:track:47qYqGPgMTh3l1PRKBHEfI' # (eletronic) Unity | FatRat

piano_music = 'spotify:track:3ekVOvu8iE33gEGLDq9lxJ' # (piano) Fantasia in D Minor | K.397




In [6]:
# guarda o id e nome
music_ids = tracks[['name', 'artists']]

# remove colunas que não serão utilizadas
tracks.drop(['name', 'artists', 'explicit', 'time_signature', 'mode', 'key', 'duration_ms', 'popularity'], axis=1, inplace=True)
music_ids

,name,artists
id,,
spotify:track:0hJkacKTAr8GZeKwcPuUk2,Yesterday's Trash,"{""(24OGO7fr96L0azkG3pZZa2,\""Modern Life Is War..."
spotify:track:1xjFMVi8pn5VIWiqrdxAMc,promise [interlude],"{""(5b0XQwyoJBFd3MwL2YxEPO,Nohidea)""}"
spotify:track:2ahiPTFTfIPs7UKqjKhJkc,Requiem - Alex Vidal Remix,"{""(6WrPC9g41hy7LUnpxdy6u7,\""Alex Vidal\"")"",""(2..."
spotify:track:2z5xCMFxAo7dn6zsu6UwL9,Kaké,"{""(44DhAhLWsSFvSqVXDUruVE,\""Young Paris\"")""}"
spotify:track:16axQgz02GFMiU2xKMT4qJ,F the Bull$h1t - Original Mix,"{""(2ohlvFf9PBsDELdRstPtlP,\""Ferry Corsten\"")""}"
...,...,...
spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,Waking Up The Dawn,"{""(7beSXA7nBwZ5xJ8jUa77ZL,\""Michael English\"")""}"
spotify:track:1nvTse7nkCr8tO8LOAell4,Here Comes the Star,"{""(48YxSlb23RAaCd4RyHcV9V,\""Herman's Hermits\"")""}"
spotify:track:5vKVjyFJNtfdHXzzLaWbpz,You're With Me,"{""(0WTXG5dSB0wpykAqPpEmFl,UnchainD)""}"


In [7]:
tracks

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,
spotify:track:0hJkacKTAr8GZeKwcPuUk2,0.004530,0.324,0.971,0.012800,0.3340,-3.168,0.0548,95.721
spotify:track:1xjFMVi8pn5VIWiqrdxAMc,0.833000,0.545,0.495,0.629000,0.3860,-13.538,0.1020,62.631
spotify:track:2ahiPTFTfIPs7UKqjKhJkc,0.033200,0.695,0.592,0.892000,0.5790,-9.588,0.0345,125.008
spotify:track:2z5xCMFxAo7dn6zsu6UwL9,0.000440,0.450,0.629,0.895000,0.1080,-7.059,0.1460,193.228
spotify:track:16axQgz02GFMiU2xKMT4qJ,0.000927,0.846,0.707,0.903000,0.0666,-5.949,0.1270,127.970
...,...,...,...,...,...,...,...,...
spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,0.109000,0.538,0.609,0.000006,0.0948,-7.392,0.0275,87.960
spotify:track:1nvTse7nkCr8tO8LOAell4,0.698000,0.339,0.325,0.000005,0.5550,-11.445,0.0252,91.286
spotify:track:5vKVjyFJNtfdHXzzLaWbpz,0.018600,0.482,0.443,0.000345,0.1100,-8.622,0.0276,144.962


### Função de normalização usando fórmula do MinMaxScaler do SKLearn 
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [8]:
def scaler(X, min_=None, max_=None):
    if not min_:
        min_ = X.min()
    if not max_:
        max_ = X.max()
    return X.map(lambda x: (x - min_) / (max_ - min_))

### Normalização dos dados que não estão normalizados

In [9]:
#tracks['duration_ms'] = scaler(tracks['duration_ms'])
#tracks['popularity'] = scaler(tracks['popularity'], min_=0, max_=100)
#tracks['key'] = scaler(tracks['key'])
tracks['loudness'] = scaler(tracks['loudness'], min_=-60, max_=0)
tracks['tempo'] = scaler(tracks['tempo'])

In [10]:
tracks

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,
spotify:track:0hJkacKTAr8GZeKwcPuUk2,0.004530,0.324,0.971,0.012800,0.3340,0.875375,0.0548,0.382904
spotify:track:1xjFMVi8pn5VIWiqrdxAMc,0.833000,0.545,0.495,0.629000,0.3860,0.715648,0.1020,0.250537
spotify:track:2ahiPTFTfIPs7UKqjKhJkc,0.033200,0.695,0.592,0.892000,0.5790,0.776489,0.0345,0.500058
spotify:track:2z5xCMFxAo7dn6zsu6UwL9,0.000440,0.450,0.629,0.895000,0.1080,0.815443,0.1460,0.772952
spotify:track:16axQgz02GFMiU2xKMT4qJ,0.000927,0.846,0.707,0.903000,0.0666,0.832540,0.1270,0.511907
...,...,...,...,...,...,...,...,...
spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,0.109000,0.538,0.609,0.000006,0.0948,0.810314,0.0275,0.351858
spotify:track:1nvTse7nkCr8tO8LOAell4,0.698000,0.339,0.325,0.000005,0.5550,0.747886,0.0252,0.365163
spotify:track:5vKVjyFJNtfdHXzzLaWbpz,0.018600,0.482,0.443,0.000345,0.1100,0.791368,0.0276,0.579878


## Criando Modelo de Similaridade

In [11]:
class Similarity:
    def __init__(self, distance='euclidean'):
        self.distance = distance
        self.tracks = None
        
    def fit(self, tracks, id_tracks):
        self.tracks = tracks
        self.id_tracks = id_tracks
    
    def recommend(self, user_tracks, n_tracks=5):
        # cria a música média
        mean_track = np.mean(user_tracks, axis=0)
        
        # calcula distancias e pega os indices das k músicas com menores distâncias
        dists = cdist([mean_track], self.tracks, self.distance)[0]
        k_arg_dists = np.argpartition(dists, n_tracks+1)[:n_tracks+1]
        
        k_ids = self.id_tracks.iloc[k_arg_dists]
        k_info = self.tracks.iloc[k_arg_dists]
        k_dists = pd.DataFrame(dists[k_arg_dists], columns=['Distâncias'], index=self.tracks.index[k_arg_dists])
        
        # retorna um dataframe com informação do nome, artistas, distância e features ordenadas pela distância
        return pd.concat([k_dists, k_ids, k_info], axis=1).sort_values(by=['Distâncias'])
        

### Teste de Recomendador para Música Única

Tabela de Acerto

MB - Muito Bom

B - Bom

OK - Ok 

R - Ruim

MR - Muito Ruim

/  -  Sem informação

In [18]:
sim = Similarity(distance='euclidean')
sim.fit(tracks, music_ids)
sim.recommend([tracks.loc[classical_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:4Znak3UnsXjRYQDSK0oP6V,0.000000,"Symphony No.5 in C minor, Op.67: 1. Allegro co...","{""(2wOqMjp9TyABvtHdOSOTUS,\""Ludwig van Beethov...",0.900,0.309,0.307,0.745,0.0929,0.764198,0.0443,0.402445
spotify:track:061pN2sHhLNI3vAPIHVn2R,0.053293,Stranger On The Shore,"{""(5bdgb81IJMXag724quaNSA,\""Acker Bilk\"")"",""(4...",0.940,0.319,0.322,0.742,0.1040,0.740200,0.0359,0.390764
spotify:track:1pKwem2d9YXLd4LUParsid,0.055306,THEME FROM LOVE STORY,"{""(1sLpCr5xThZkGHLw4H0mNE,\""101 Strings Orches...",0.900,0.289,0.315,0.716,0.1050,0.728632,0.0288,0.412557
spotify:track:1mBIJRcbAec5EkveeYrPCa,0.065940,I Got Lost,"{""(267VY6GX5LyU5c9M85ECZQ,\""Dinosaur Jr.\"")""}",0.875,0.354,0.330,0.748,0.0942,0.738983,0.0264,0.388224
spotify:track:5GA90eh24Ro95k9tywfuHo,0.069033,Unless a Grain of Wheat,"{""(3PF6OyABdvKssByiQ8Imn9,\""Forbes Ministry of...",0.886,0.359,0.292,0.738,0.1310,0.756327,0.0276,0.404209
spotify:track:42B96jvtflrB3Xo7rcJnl2,0.069542,American Civil War Fantasy,"{""(6lxjjQH0jBo0mEHO5WxFnS,\""United States Coas...",0.910,0.319,0.303,0.773,0.0794,0.741124,0.0388,0.458044


OK, MB, R, R, B 

In [19]:
sim.recommend([tracks.loc[rap_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:5Tou8lYopj9qCIFBFBomZ5,0.000000,Yoshi City,"{""(67lytN32YpUxiSeWlKfHJ3,\""Yung Lean\"")""}",0.476,0.546,0.530,0.000000,0.0955,0.784637,0.211,0.512863
spotify:track:0kDEzhgtVZfYMVm7VR7f03,0.060607,Hero,"{""(0fLP4yWtS7wE0S0QbI4IYG,\""Chan Dizzy\"")""}",0.436,0.563,0.539,0.000000,0.1220,0.796189,0.232,0.492210
spotify:track:2CIzsTwVjIWMh64Nw6S3Lp,0.061630,Heartbroken Forever,"{""(5WF5qT6fSfyd3RC2p8qpEX,\""Tender Forever\"")""}",0.518,0.534,0.509,0.000000,0.1130,0.786131,0.217,0.479625
spotify:track:4vh8S4hTWAz3DHjysBo0ai,0.061667,Cocaine,"{""(4YUcg2ee9TxMNdjhAxzZBF,Ashh)"",""(44JOYPmzHzT...",0.498,0.584,0.506,0.000028,0.1110,0.772084,0.181,0.511899
spotify:track:4va49u7BqBLAmpu7WnyuQ1,0.064300,Heartbreaker,"{""(4mwXUEKaW4ftbncf9Hi58l,Tank)""}",0.496,0.510,0.500,0.000000,0.0637,0.805677,0.217,0.519827
spotify:track:00VWUa96Clte4uNEv963vn,0.070310,Évite la,"{""(1cA379ILgR07m7XQVXyP6Q,Sneazzy)"",""(5olcgTze...",0.442,0.603,0.538,0.000049,0.1010,0.786347,0.232,0.513463


B, B, MB, R, B

In [20]:
sim.recommend([tracks.loc[rap_music2]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:15TJBhID2NsMMp2etO0B03,0.000000,Wrist Cry,"{""(2xvtxDNInKDV4AvGmjw6d1,Bladee)""}",0.366,0.634,0.428,0.003270,0.280,0.738906,0.0284,0.439895
spotify:track:5X7yF7u1fVGS4EyekX6ewM,0.025748,"Miami, My Amy","{""(0bc7dhY6YXL35m1G0jVwuW,\""Keith Whitley\"")""}",0.354,0.646,0.441,0.000002,0.290,0.746592,0.0272,0.433999
spotify:track:1OJpQrusmejmwnvsFQqF8c,0.035643,Annie,"{""(05Pp7SeRkvE236OxUF2qmo,Yarn)""}",0.341,0.645,0.443,0.000027,0.290,0.752599,0.0276,0.440767
spotify:track:3zyUs7o9iKVhb1xuSSi0t8,0.038462,Compañero de Mi Alma,"{""(3NUe53SAgcR19GtgfSKTpW,\""Óscar Medina\"")""}",0.374,0.646,0.413,0.000000,0.265,0.711628,0.0302,0.431942
spotify:track:0Avs2CCYWfhanvCMc5qtnK,0.040308,Friends - Acoustic,"{""(6jO0ArPsbU6gg4GehuCjoF,\""Eliza and the Bear...",0.331,0.633,0.429,0.000006,0.299,0.743912,0.0293,0.440395
spotify:track:0DT4o6GdCw5Uxqb5EDIqSf,0.043744,Don't Try To Explain,"{""(6iDaoPZVgxrTkndDCisX8F,\""Keb' Mo'\"")""}",0.386,0.649,0.452,0.000469,0.278,0.762904,0.0393,0.442339


MR, MR, MR, /, MR

In [21]:
sim.recommend([tracks.loc[anime_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:0zFRHGYU1zP6q45qbils3z,0.000000,GO!!!,"{""(3w2HqkKa6upwuXEULtGvnY,FLOW)""}",0.0508,0.604,0.978,0.000000,0.228,0.891595,0.0573,0.536584
spotify:track:5X31etfs08Q3H7WenRHWoc,0.038166,UNBALANCE,"{""(7Jo4alROJu9Smj8IZahiP1,Dios/signalP)""}",0.0471,0.590,0.956,0.004540,0.242,0.881752,0.0395,0.548101
spotify:track:5XuGyBKKFrCoz4iwvm7k7s,0.042463,I Hope,"{""(6KhH771vq2X2Aom91nNzpZ,FTISLAND)""}",0.0580,0.590,0.976,0.000000,0.197,0.892426,0.0641,0.559893
spotify:track:5z8ynRCdK17BWNLxL8rrfZ,0.043371,Take It Like a Man,"{""(72OaDtakiy6yFqkt4TsiFt,Cher)""}",0.0678,0.613,0.950,0.000000,0.243,0.889977,0.0723,0.520019
spotify:track:62pM9hM5FdqPuUKkZTmENp,0.043751,fanfare,"{""(0udR1QMfgT37SQnL1QClTU,\""Bullet Train\"")""}",0.0426,0.613,0.943,0.000000,0.208,0.883323,0.0651,0.540000
spotify:track:4618Zrqjfr2nqeXibDUIFJ,0.044536,Make me fly - The Out In Space Story Continues,"{""(40edmg1pVhEKklKU7BCo1n,\""Timur Rodriguez\"")""}",0.0263,0.620,0.967,0.000008,0.250,0.889069,0.0475,0.516095


MB, MB, R, MB, R

In [22]:
sim.recommend([tracks.loc[rock_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:2hmibAtdObO8F4tnhLENuQ,0.000000,Reptilia,"{""(0epOFNiUfyON9EYx7Tpr6V,\""The Strokes\"")""}",0.000597,0.488,0.650,0.714,0.1010,0.845463,0.0336,0.632077
spotify:track:79NvOn5z1gmBhZMvAdaZzv,0.039079,L'innocent,"{""(0iF2gedsSjoVHRUh5vXSdu,\""Cirque du Soleil\"")""}",0.013300,0.496,0.638,0.687,0.0973,0.827134,0.0296,0.640021
spotify:track:1Dc4H1XvXdsRqrD6WZxYKZ,0.056278,Lower Down,"{""(4zxyqU3xB0RrPX9rL9RNln,\""Lo Tom\"")""}",0.000043,0.483,0.699,0.714,0.1210,0.853781,0.0292,0.616192
spotify:track:6LVbyUVhbhhNN7uc2bqEqg,0.060271,Strip The Soul,"{""(5NXHXK6hOCotCF8lvGM1I0,\""Porcupine Tree\"")""}",0.001920,0.537,0.670,0.716,0.1150,0.830538,0.0470,0.616992
spotify:track:3JxGBrNcvunLiznL1JJzZN,0.066181,Is This Real?,"{""(0sTTw3dw3EA0c7NaZnrJd2,Wipers)""}",0.003300,0.456,0.695,0.714,0.1310,0.839302,0.0324,0.651678
spotify:track:1QlweZy3kMgha3UlkQbjMS,0.066457,Take This Lying Down,"{""(2VS0tRcCbiMkelzGoZo9XP,\""The Lucksmiths\"")""}",0.002320,0.479,0.631,0.711,0.0732,0.797268,0.0264,0.603527


OK, B, B, B, R

In [23]:
sim.recommend([tracks.loc[eletronic_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:47qYqGPgMTh3l1PRKBHEfI,0.000000,Unity,"{""(3OKg7YbOIatODzkRIbLJR4,TheFatRat)""}",0.004240,0.707,0.880,0.03790,0.0534,0.874713,0.0317,0.420034
spotify:track:6Gd317x1MJZ378T6xbCpGI,0.041626,Jacuzzi Jam - Original Mix,"{""(3uMZB1LatgHJCRe7DBtgiK,Syndaesia)""}",0.000565,0.728,0.873,0.03190,0.0659,0.872125,0.0569,0.439915
spotify:track:5HRnngZD7Xgko5swVsuYIk,0.043683,How I Feel,"{""(1fwbNA9Ymk7hPxcxx67paW,\""Brown and Gammon\"")""}",0.002050,0.698,0.889,0.03740,0.0418,0.856692,0.0541,0.447963
spotify:track:3MYStY0UdBrk01XUDfYON1,0.046224,Phoenix - DJ Laszlo Remix,"{""(188VINPQh5dyNfLUkevKqf,\""Olivia Holt\"")"",""(...",0.029200,0.697,0.899,0.01050,0.0407,0.881999,0.0365,0.412001
spotify:track:6DlwyiBUy2b0JQWd6CLAQa,0.046876,Love To Love You Baby - Giorgio Moroder Feat. ...,"{""(2eogQKWWoohI3BSnoG7E2U,\""Donna Summer\"")"",""...",0.019000,0.710,0.878,0.00499,0.0623,0.853303,0.0458,0.432158
spotify:track:5TYIzi4Oa1fiRpDX0mklLN,0.047079,Hunter - Made in June Remix,"{""(4sTQVOfp9vEMCemLw50sbu,Galantis)"",""(0EJYJn2...",0.005150,0.719,0.865,0.00709,0.0827,0.871987,0.0356,0.424062


MB, MB, B, MB, B

In [24]:
sim.recommend([tracks.loc[piano_music]])

,Distâncias,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,,,
spotify:track:3ekVOvu8iE33gEGLDq9lxJ,0.000000,"Fantasia in D Minor, K.397","{""(4NJhFmfw43RLBLjQvxDuRS,\""Wolfgang Amadeus M...",0.993,0.392,0.02490,0.946,0.1140,0.482341,0.0632,0.539516
spotify:track:2CUFmzZhLz1DjE0jPSqsGt,0.030968,"Livre de guitarre dédié au Roy, Suite No. 3 en...","{""(4br5HVmW3Xed5Zvffl0RSU,\""Robert de Visée\"")...",0.993,0.371,0.03750,0.944,0.1080,0.475610,0.0588,0.555473
spotify:track:2tQsfOpi7hC0mUvHLZ00W6,0.032242,"Sentimental Waltz, Op. 51, No. 6: Sentimental ...","{""(3KA4Qo2Ngnk5knbCkiowDI,\""Daria Telizyn\"")""}",0.995,0.395,0.00402,0.930,0.1220,0.484528,0.0469,0.539808
spotify:track:1X5Qxd5vVlvOoVYVJCfAwL,0.032418,Metamorphosis I (arr. for harp),"{""(1xWAOwW7fYZprzRMvR2v9Z,Anonymous )"",""(69lxx...",0.993,0.401,0.03060,0.939,0.1090,0.492553,0.0881,0.551313
spotify:track:28JqEjN2SNFk8NqHQfnvxB,0.032968,"On an Overgrown Path, Pt. 1: IV. The Madonna o...","{""(46xXzoOdtD7SL2PTRq5irp,\""Leoš Janáček\"")"",""...",0.993,0.390,0.00248,0.949,0.0919,0.481909,0.0563,0.533600
spotify:track:2HkJfFiGyBsIu0xWNDER3H,0.033162,"Mazurka No. 21 in C-Sharp Minor, Op. 30, No. 4","{""(7y97mc3bZRFXzT2szRM4L4,\""Frédéric Chopin\"")...",0.994,0.401,0.00789,0.930,0.0969,0.479845,0.0512,0.534060


B, B, B, MB, MB

### Teste de Recomendador para Playlist de Músicas

In [ ]:
class ContentBasedEvaluator:
    '''
    Esse Evaluator pretende avaliar o recomendador.
    Recebe um modelo, recomenda para as tracks de treino e retorna as distâncias que esse modelo encontrou
    para os dados de teste. Então, se temos duas músicas que consideramos parecidas, podemos colocar
    uma para ser de treino e outra de teste e então ele vai retornar a distância delas. Se for pequena isso
    mostra que o modelo percebeu similaridade
    '''
    def __init__(self, fitted_model, tracks_size):
        self.tracks_size = tracks_size
        self.fitted_model = fitted_model
        
    def evaluate(self, train_tracks, test_tracks):
        recommendations = self.fitted_model.recommend(train_tracks, n_tracks=self.tracks_size-2)
        return recommendations[recommendations.index.isin(test_tracks.index)]
    

In [ ]:
cbe = ContentBasedEvaluator(sim4, len(tracks))
cbe.evaluate(
    pd.DataFrame([tracks.loc[anime_music]]), 
    pd.DataFrame([tracks.loc[classical_music_i]]))

In [ ]:
dataset = SpotifyPlaylistDataset()
dataset.read_playlist_formatted(100000)

# Modelo de Recomendação Content Based com Clusterização

In [ ]:
class SimilarityMeans:
    def __init__(self, distance='euclidean'):
        self.distance = distance
        self.tracks = None
        self.id_tracks = None
        
    def fit(self, tracks, id_tracks):
        self.tracks = tracks
        self.id_tracks = id_tracks
        
    def clusterfy(self, user_tracks):
        km = KMeans()
        visualizer = KElbowVisualizer(km, k=(1,len(user_tracks)+1))
        visualizer.fit(user_tracks)
        
        elbow_v = visualizer.elbow_value_
        new_km = KMeans(n_clusters=elbow_v)
        new_km.fit(user_tracks)
        
        return new_km
    
    def recommend_clusters(self, user_tracks, n_tracks=5):
        self.tracks = self.tracks.drop(index=user_tracks.index)
        self.id_tracks = self.id_tracks.drop(index=user_tracks.index)
        
        if len(user_tracks)==1:
            return [self.recommend(user_tracks[0], n_tracks)]
        
        model = self.clusterfy(user_tracks)
        clusters = model.cluster_centers_
        tracks = []
        for i in range(len(clusters)) :
            rec = self.recommend(clusters[i], n_tracks)
            tracks.append(rec)
        
        return tracks, pd.Series(model.labels_), clusters
        
    def recommend(self, user_track, n_tracks=5):
        # calcula distancias e pega os indices das k músicas com menores distâncias
        dists = cdist([user_track], self.tracks, self.distance)[0]
        k_arg_dists = np.argpartition(dists, n_tracks)[:n_tracks]
        
        k_ids = self.id_tracks.iloc[k_arg_dists]
        k_info = self.tracks.iloc[k_arg_dists]
        k_dists = pd.DataFrame(dists[k_arg_dists], columns=['Distâncias'], index=self.tracks.index[k_arg_dists])
        
        # retorna um dataframe com informação do nome, artistas, distância e features ordenadas pela distância
        return pd.concat([k_dists, k_ids, k_info], axis=1).sort_values(by=['Distâncias'])

In [ ]:
rec = SimilarityMeans(distance='euclidean')
rec.fit(tracks, music_ids)

In [ ]:
tracks_to_recommend = tracks.loc[p1]
recommended, labels, centroids = rec.recommend_clusters(tracks_to_recommend)

In [ ]:
for i,r in enumerate(recommended):
    print(f'Músicas pertecentes ao cluster {i+1}:')
    display(music_ids.loc[tracks_to_recommend[np.array(labels==i)].index])
    print(f'Recomendação para o cluster {i+1}:')
    display(r)

In [ ]:
large_playlist = dataset.read_playlist_formatted(2000)['track_uri']
large_playlist

In [ ]:
rec_large = SimilarityMeans(distance='euclidean')
rec_large.fit(tracks, music_ids)

In [ ]:
tracks_to_recommend = tracks.loc[large_playlist]
recommended, labels, centroids = rec_large.recommend_clusters(tracks_to_recommend)
print('Quantidade de clusters:', len(centroids))
for i,r in enumerate(recommended):
    print(f'Músicas pertecentes ao cluster {i+1}:')
    display(music_ids.loc[tracks_to_recommend[np.array(labels==i)].index])
    print(f'Recomendação para o cluster {i+1}:')
    display(r)